In [31]:
### resdata - Jupyter Notebook analysis pipeline test
## The testing cells.
#1. Fetch single ring by id and visualise the data
#2. Fetch relevant data from full experiment and visualise as histogram
#3. Fetch relevant data from full experiment and visualise

In [13]:
## Sing in and Authentication
username_init = 'test_user_main'
password_init = 'some_password'
email = 'email@test_email.com'
full_name = 'Tomasz Neska'
path = 'http://127.0.0.1:8000/'
file_name = "test.json"
# declaration of variables used

In [14]:
# initialisation cell
import sys
sys.path.insert(0, "/home/splitsky/Desktop/code_repositories/resdata/")
import interface as ui
api = ui.API_interface(path)
api.check_connection()
# purge call - comment out if necessary -> just for development
api.purge_everything()

purged


In [15]:
# make user and populate the database with data
result = api.create_user(username_in=username, password_in=password,email=email,full_name=full_name)
print("User created "  + str(result))

# populate the database
no_of_rings = 2
no_of_experiments = 2

import testing as t
api.generate_token(username, password)
project_name = "project_test_"
experiment_name = "experiment_test"
ds_size = 2000
for i in range(0,4,1):
    t.generate_optics_project(file_name,[no_of_rings, no_of_experiments], project_name+str(i), experiment_name, author_name=username, size_of_dataset=ds_size)
    project = t.load_file_project(file_name)
    api.insert_project(project)
    
print("Database populated")
print("")
api.tree_print()
    
    
    

User created True
Database populated

The data tree:
project_test_0
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_test 0
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
project_test_1
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_te

In [17]:
# Authenticate user
username = username_init
password = password_init
api.generate_token(username, password)

In [24]:
# Tree print
api.tree_print()

The data tree:
project_test_0
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_test 0
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
project_test_1
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_test 0
         -->ring_no. 0
         -

In [ ]:
# Pull individual data set by name

dataset = api.return_full_dataset("project_test_3","experiment_test 0", "PL spectrum - ring no.0")
x, y, ey = dataset.data
### plot etc...

import matplotlib.pyplot as plt

plt.plot(x,y)

# note: couldn't get matplot lib to work but it's all coded as if it does. May be errors
#plt.errorbar(x,y,ey)

In [44]:
# Pull single ring -> plot dimensions and all spectra
import sys
sys.path.insert(0, "/home/splitsky/Desktop/code_repositories/resdata/")
import interface as ui
#from jupyter_driver import plot_from_dataset

api = ui.API_interface(path)
api.check_connection()

api.generate_token(username, password)

api.tree_print()
ring_id = 0
experiment_id = "experiment_test 1"
project_id = "project_test_0"
projects = api.experiment_search_meta(meta_search={"ring_id": ring_id}, experiment_id=experiment_id, project_id=project_id)

for entry in projects:
    print(entry.name)

The data tree:
project_test_0
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_test 0
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
project_test_1
     ->experiment_test 1
         -->ring_no. 0
         -->PL spectrum - ring no.0
         -->TRPL spectrum - ring no.0
         -->Lasing spectrum - ring no.0
         -->ring_no. 1
         -->PL spectrum - ring no.1
         -->TRPL spectrum - ring no.1
         -->Lasing spectrum - ring no.1
     ->experiment_test 0
         -->ring_no. 0
         -

In [ ]:
# Pull all rings from experiment and summarise each

#TODO: finish this up once matplot lib works

In [ ]:
# Create a group and pull it together to summarise ring dimensions

#TODO: to implement as group functions still don't fully work

In [ ]:
# Pull group together to summarise spectra 

# same as above